# Retrieval expérimentation

## 0. Setting up

### 0.1. Import

In [37]:
import os
import random 
import csv

In [38]:
import pandas as pd

In [39]:
from transformers import pipeline

### 0.2. Paths

In [40]:
chunks_folder_path = '../data/chunks'
chunks_dataset_csv_file_path = '../data/csv/validation_template.csv'
qa_dataset_csv_file_path = '../data/csv/validation_auto_qg.csv'

## 1.Create a data set for Question answering testing

### 1.1. A bunch of chunks dataset

#### commentaire : resort la liste entière de tout mes chunks

In [41]:
chunks = sorted(os.listdir(chunks_folder_path))
nb_chunks = len(chunks)

#### commentaire: 
-  on prends le 1/10 **`au hasard`** des chunk comme pour du testing classique
- avoir plus tard si une analyse sur la distribution des données et des thématique ne pourrais pas aidé

In [42]:
sample = random.sample(chunks, int(nb_chunks/10))

#### commentaire: génration du fichier csv

In [53]:
with open(chunks_dataset_csv_file_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['chunk_id', 'text', 'question', 'reference_answer'])
    for fname in sample:
        text = open(os.path.join(chunks_folder_path, fname), encoding='utf-8').read()
        snippet = text.replace('\n', ' ') + '…'
        writer.writerow([fname, snippet, '', ''])
print(f"fichier {chunks_dataset_csv_file_path} généré avec {len(sample)} chunks.")

fichier ../data/csv/validation_template.csv généré avec 153 chunks.


### 1.2. Generations des questions

##### 1.2.1. QG avec un modèle pré-entrainé

In [49]:
model_qa_1, tokenizer_qa_1 = "valhalla/t5-base-e2e-qg", "valhalla/t5-base-e2e-qg"
model_qa_2, tokenizer_qa_2 = "t5-small", "t5-small"

In [50]:
# Pipeline end-to-end QG
qg = pipeline(
    "text2text-generation",
    model=model_qa_2,
    tokenizer=tokenizer_qa_2
)

c:\Users\regis\Documents\aivancity\pge4\clinique\chatbot\venv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\regis\Documents\aivancity\pge4\clinique\chatbot\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\regis\.cache\huggingface\hub\models--t5-small. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate De

In [54]:
df = pd.read_csv(chunks_dataset_csv_file_path, encoding='utf-8')

In [55]:
df.columns

Index(['chunk_id', 'text', 'question', 'reference_answer'], dtype='object')

In [56]:
rows = []
for chunk_id, snippet, _, _ in df.itertuples(index=False):
    # Prompt plus simple et direct
    prompt = f"generate question: {snippet}"
    
    # Paramètres ajustés pour de meilleures générations
    out = qg(
        prompt,
        max_length=100,  # Augmenté pour avoir plus de contexte
        min_length=10,   # Longueur minimale
        num_beams=5,     # Plus de beams pour de meilleurs résultats
        num_return_sequences=1,
        early_stopping=True,
        do_sample=True,  # Ajout de sampling
        temperature=0.7, # Contrôle de la créativité
        clean_up_tokenization_spaces=True
    )[0]['generated_text'].strip()

    print(f"PROMPT: {prompt[:100]}...")
    print(f"OUTPUT: {out}")
    print("-" * 50)
    
    # Parsing amélioré
    question = out
    if not question.endswith('?'):
        question += '?'
    
    # Génération de la réponse séparément
    answer_prompt = f"answer the question based on context: {question} Context: {snippet}"
    answer_out = qg(
        answer_prompt,
        max_length=80,
        num_beams=3,
        num_return_sequences=1,
        early_stopping=True
    )[0]['generated_text'].strip()
    
    if question and len(question) > 5:
        rows.append({
            'chunk_id': chunk_id,
            'question': question,
            'reference_answer': answer_out
        })
    
    break  # pour test

# 5. Construire et afficher le DataFrame résultat
df_out = pd.DataFrame(rows)
print("Exemple QA générée :", df_out.dropna().iloc[0][['question','reference_answer']].to_dict())

PROMPT: generate question: real-world, in-home environments, as this is where most seizures progressinthefie...
OUTPUT: non-EEG datastrictlyforneurologicalresearchanddoesnotstoreanynon-EEG detectionsystemscanbemodelledtoaccuratelyreflectthereal-world resources
--------------------------------------------------
Exemple QA générée : {'question': 'non-EEG datastrictlyforneurologicalresearchanddoesnotstoreanynon-EEG detectionsystemscanbemodelledtoaccuratelyreflectthereal-world resources?', 'reference_answer': 'complexities encountered by PWE in their day to day lives'}


##### 1.2.2. mistral

In [60]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from dotenv import load_dotenv

In [61]:
load_dotenv()
HF_TOKEN =os.getenv('HUGGINGFACE_API_TOKEN')

In [ ]:
print("Chargement de Mistral avec token...")
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

# Chargement avec token
tokenizer = AutoTokenizer.from_pretrained(model_name, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=HF_TOKEN,
    torch_dtype=torch.float16,
    device_map="auto"
)

mistral = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

print("✅ Mistral chargé avec succès!")

Chargement de Mistral...


c:\Users\regis\Documents\aivancity\pge4\clinique\chatbot\venv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [ ]:
# 2. Fonction simple pour générer Q&A
def generer_qa(texte_medical):
    prompt = f"""[INST] Tu es un médecin spécialisé en épilepsie. 
Lis ce texte et créé UNE question et sa réponse.

Texte: {texte_medical}

Réponds exactement dans ce format:
Q: [ta question]
R: [ta réponse]
[/INST]"""

    response = mistral(
        prompt,
        max_new_tokens=120,
        temperature=0.7,
        do_sample=True,
        return_full_text=False
    )
    
    texte_genere = response[0]['generated_text']
    
    # Extraction simple
    lignes = texte_genere.split('\n')
    question = None
    reponse = None
    
    for ligne in lignes:
        if ligne.startswith('Q:'):
            question = ligne.replace('Q:', '').strip()
        elif ligne.startswith('R:'):
            reponse = ligne.replace('R:', '').strip()
    
    return question, reponse

In [ ]:
# 3. Test sur vos chunks
rows = []

# Traitement de chaque chunk
for chunk_id, snippet, _, _ in df.itertuples(index=False):
    print(f"\nTraitement chunk {chunk_id}...")
    print(f"Texte: {snippet[:100]}...")
    
    question, reponse = generer_qa(snippet)
    
    if question and reponse:
        rows.append({
            'chunk_id': chunk_id,
            'question': question,
            'reference_answer': reponse
        })
        print(f"✅ Q: {question}")
        print(f"✅ R: {reponse}")
    else:
        print("❌ Échec génération")
    
    # Test sur 3 chunks seulement
    if len(rows) >= 3:
        break

# 4. Résultat final
df_out = pd.DataFrame(rows)
print(f"\n🎉 {len(df_out)} Q&A générées!")

if not df_out.empty:
    print("\nExemple final:")
    exemple = df_out.iloc[0]
    print(f"Question: {exemple['question']}")
    print(f"Réponse: {exemple['reference_answer']}")